In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from typing import List

In [2]:
caminho_base = '../data/processed/df_processed_v2.parquet'

In [3]:
base_completa = pd.read_parquet(caminho_base)

In [4]:
base_completa

,NOME_AEROPORTO_ORIGEM,CIDADE_AEROPORTO_ORIGEM,UF_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NUMERO_VOO,CODIGO_TIPO_LINHA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_AEROPORTO_DESTINO,PARTIDA_PREVISTA,PARTIDA_REAL,...,INDICE_SEMANA_CHEGADA_REAL,INDICE_SEMANA_CHEGADA_PREVISTA,FAIXA_ATRASO_PARTIDA,FAIXA_ATRASO_CHEGADA,FLAG_ATRASO_PARTIDA,FLAG_ATRASO_CHEGADA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,FLAG_FIM_SEMANA_PARTIDA_REAL,FLAG_FIM_SEMANA_CHEGADA_PREVISTA,FLAG_FIM_SEMANA_CHEGADA_REAL
0,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-02 00:05:00,2022-01-01 23:35:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
1,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-09 01:00:00,2022-01-09 00:48:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
2,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-23 11:05:00,2022-01-23 10:54:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
3,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-30 11:05:00,2022-01-30 11:01:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
4,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-02-26 23:00:00,2022-02-26 22:56:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717352,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-15 14:34:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717353,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-18 15:35:00,2022-12-18 15:41:00,...,6,6,Atraso de até 30 minutos,No horário,Vôo no horário,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
717354,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-22 14:46:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717355,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-25 15:35:00,2022-12-25 15:11:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana


- Situação Vôo

In [5]:
def obter_situacao_voo(
        sigla_aeroporto: str,
        mes_partida_prevista: List[int],
        codigo_tipo_linha: str, 
        sigla_empresa: str = None,
       
):
    query = (
            f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f' and MES_PARTIDA_PREVISTA in {mes_partida_prevista}'  # TAM SBRP 1
            f' and CODIGO_TIPO_LINHA == "{codigo_tipo_linha}"'
    )
    colunas = [
            'SITUACAO_VOO', 
            'MES_PARTIDA_PREVISTA', 
            'NOME_MES_PARTIDA_PREVISTA', 
            'SIGLA_ICAO_AEROPORTO_ORIGEM',
            'CODIGO_TIPO_LINHA'
    ]

    if sigla_empresa is not None:
        query += f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')
   
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    total = dataframe.query(query)[colunas].value_counts()
    percentual = dataframe.query(query)[colunas].value_counts( normalize=True,) * 100
    dataframe = pd.concat([total, percentual], axis=1).reset_index()
    dataframe['TOTAL_SITUACAO'] = dataframe['count'].astype('int32')
    dataframe['PROPORCAO'] = dataframe['proportion'].astype('float32')
    dataframe = dataframe.sort_values(by=['SITUACAO_VOO', 'MES_PARTIDA_PREVISTA']) 
    dataframe.drop(['proportion', 'count', 'CODIGO_TIPO_LINHA'], axis=1, inplace=True,)
    dataframe['SITUACAO_VOO'] =  dataframe['SITUACAO_VOO'].astype('string')
    if sigla_empresa is not None:
        dataframe = dataframe.sort_values(by='MES_PARTIDA_PREVISTA')
    return dataframe 

In [6]:
dataframe = obter_situacao_voo(
    sigla_aeroporto='SBRP', 
    mes_partida_prevista=[i for i in range(1, 13)], # [1,2]
    codigo_tipo_linha='N',
    sigla_empresa='TAM'
)
dataframe

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,TOTAL_SITUACAO,PROPORCAO
2,REALIZADO,1,janeiro,SBRP,TAM,41,11.781610
10,REALIZADO,2,fevereiro,SBRP,TAM,20,5.747127
4,REALIZADO,3,março,SBRP,TAM,23,6.609195
8,REALIZADO,4,abril,SBRP,TAM,21,6.034483
11,REALIZADO,5,maio,SBRP,TAM,20,5.747127
7,REALIZADO,6,junho,SBRP,TAM,22,6.321839
9,REALIZADO,7,julho,SBRP,TAM,21,6.034483
5,REALIZADO,8,agosto,SBRP,TAM,23,6.609195
6,REALIZADO,9,setembro,SBRP,TAM,23,6.609195
3,REALIZADO,10,outubro,SBRP,TAM,24,6.896552


In [7]:
dataframe['SITUACAO_VOO'].unique()

<StringArray>
['REALIZADO', 'CANCELADO']
Length: 2, dtype: string

In [8]:
fig = px.bar(
    dataframe,
    x='NOME_MES_PARTIDA_PREVISTA',
    y='TOTAL_SITUACAO',
    color='SITUACAO_VOO',
    barmode='group',
    text_auto='0'
)

fig.show()

- Variação  da Situação do Voo em relação ao mês anterior (Não expressar em porcentagem)

In [9]:
def obter_variacao_situacao_voo(sigla_aeroporto: str, mes_partida_prevista: List[int], codigo_tipo_linha: str, situacao_voo: str,  sigla_empresa: str = None):
    dataframe = obter_situacao_voo(
        sigla_aeroporto=sigla_aeroporto, 
        mes_partida_prevista=mes_partida_prevista, # [1,2]
        codigo_tipo_linha=codigo_tipo_linha,
        sigla_empresa=sigla_empresa
    )
    dataframe.drop('PROPORCAO', axis=1, inplace=True)
    dataframe['TOTAL_SITUACAO_MES_ANTERIOR'] = dataframe.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
    dataframe.fillna(0,axis=1,inplace=True)
    dataframe = dataframe.query(f' SITUACAO_VOO == "{situacao_voo}" ')
    return dataframe

In [10]:
dataframe = obter_variacao_situacao_voo(
    sigla_aeroporto='SBRP', 
    mes_partida_prevista=[i for i in range(1, 13)], # [1,2]
    codigo_tipo_linha='N',
    situacao_voo='REALIZADO'
)
dataframe 

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR
0,REALIZADO,1,janeiro,SBRP,582,0
11,REALIZADO,2,fevereiro,SBRP,249,582.0
9,REALIZADO,3,março,SBRP,345,249.0
5,REALIZADO,4,abril,SBRP,381,345.0
1,REALIZADO,5,maio,SBRP,433,381.0
3,REALIZADO,6,junho,SBRP,387,433.0
10,REALIZADO,7,julho,SBRP,250,387.0
2,REALIZADO,8,agosto,SBRP,409,250.0
7,REALIZADO,9,setembro,SBRP,368,409.0
6,REALIZADO,10,outubro,SBRP,375,368.0


In [11]:
fig = go.Figure()


fig.add_trace(
    go.Bar(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_SITUACAO'],
        name='Mês atual',

    )
)


fig.add_trace(
    go.Scatter(
        x=dataframe['NOME_MES_PARTIDA_PREVISTA'],
        y=dataframe['TOTAL_SITUACAO_MES_ANTERIOR'],
        name='Mês anterior'
    )
)
fig.update_layout(
    title='Teste'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês compare com o mês anterior (Talvez com vísual cartões) deixe filtro facultativo por empresa)

In [12]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_SITUACAO']
        total_mes_anterior = linha['TOTAL_SITUACAO_MES_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)

In [13]:
def obter_variacao_percentual(sigla_aeroporto: str, situacao_voo: str, mes_partida: int, codigo_tipo_linha: str,  sigla_empresa: str = None):
    colunas = [
        'SITUACAO_VOO',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        'CODIGO_TIPO_LINHA'
    ]
    query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" and  CODIGO_TIPO_LINHA == "{codigo_tipo_linha}" '
    if sigla_empresa is not None:
        query += f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}"  '
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')

    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]] = dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]].astype('string')
    dataframe = dataframe.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
        TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
    ).sort_values(by=['SITUACAO_VOO','MES_PARTIDA_PREVISTA']).reset_index()
    dataframe['TOTAL_SITUACAO_MES_ANTERIOR'] = dataframe.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['PERCENTUAL_VARIACAO'] = dataframe.apply(calcular_variacao, axis=1)
    query = f'SITUACAO_VOO == "{situacao_voo}" and MES_PARTIDA_PREVISTA == {mes_partida}'

    dataframe = dataframe.query(query)[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]

    return dataframe

In [14]:
dataframe = obter_variacao_percentual(
    sigla_aeroporto='SBRP',
    sigla_empresa='TAM',
    mes_partida=2, 
    situacao_voo='REALIZADO',
    codigo_tipo_linha='N'
)
dataframe

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
3,20,41.0,-51.22


In [15]:
fig = go.Figure(
    go.Indicator(
        mode='number+delta',
        value=dataframe['TOTAL_SITUACAO'].values[0],
        delta={
            'position': 'top', 
            'reference': dataframe['TOTAL_SITUACAO_MES_ANTERIOR'].values[0], 
            'relative': True,
            'valueformat': '.2%'
        },
        title={
            'text': 'Total Situação Vôos Realizados'
        },
    )
)
fig.show()

- Compare total de vôos realizado no fim de semana em relação por dia útil por mês

In [16]:
def obter_dados_voos_dia_semana(sigla_aeroporto: str, mes_partida: int, sigla_empresa: str = None):
    colunas = [
        'SITUACAO_VOO',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        'INDICE_SEMANA_PARTIDA_PREVISTA',
        'DIA_DA_SEMANA_PARTIDA_PREVISTA'
    ]
    query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" and MES_PARTIDA_PREVISTA == {mes_partida} '
    if sigla_empresa is not None:
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')
        query += f'SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '

    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[['DIA_DA_SEMANA_PARTIDA_PREVISTA','SITUACAO_VOO']] = dataframe[['DIA_DA_SEMANA_PARTIDA_PREVISTA','SITUACAO_VOO']].astype('string')
    dataframe = dataframe.groupby(['DIA_DA_SEMANA_PARTIDA_PREVISTA', 'INDICE_SEMANA_PARTIDA_PREVISTA', 'SITUACAO_VOO']).agg(
        TOTAL_VOOS=('DIA_DA_SEMANA_PARTIDA_PREVISTA', 'count')
    ).reset_index()
    dataframe = dataframe.sort_values(by=['SITUACAO_VOO','INDICE_SEMANA_PARTIDA_PREVISTA'])
    return dataframe

In [17]:
dataframe = obter_dados_voos_dia_semana(
    sigla_aeroporto='SBRP',
    mes_partida=1
)
dataframe

,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_VOOS
6,Segunda-feira,0,CANCELADO,3
12,Terça-feira,1,CANCELADO,1
2,Quarta-feira,2,CANCELADO,2
4,Quinta-feira,3,CANCELADO,5
8,Sexta-feira,4,CANCELADO,4
10,Sábado,5,CANCELADO,1
0,Domingo,6,CANCELADO,5
7,Segunda-feira,0,REALIZADO,329
13,Terça-feira,1,REALIZADO,41
3,Quarta-feira,2,REALIZADO,42


In [18]:
fig = px.bar(
    dataframe, 
    x='DIA_DA_SEMANA_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
    color='SITUACAO_VOO',
    barmode='group'
)
fig.update_traces(
    textposition='outside'
)
fig.show()

<p> Analise a FAIXA_ATRASO_PARTIDA por aeroporto e empresa por dia de semana e por mês </p>

In [19]:
colunas = [
    'FAIXA_ATRASO_PARTIDA',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA'
]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [20]:
dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "AZU" and SIGLA_ICAO_AEROPORTO_ORIGEM=="SBRP" and MES_PARTIDA_PREVISTA  == 2')
# dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "AZU" and SIGLA_ICAO_AEROPORTO_ORIGEM=="SBRP" ')

In [21]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1938 to 237113
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   FAIXA_ATRASO_PARTIDA            146 non-null    category
 1   MES_PARTIDA_PREVISTA            146 non-null    int32   
 2   NOME_MES_PARTIDA_PREVISTA       146 non-null    category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM     146 non-null    string  
 4   NOME_AEROPORTO_ORIGEM           146 non-null    string  
 5   SIGLA_ICAO_EMPRESA_AEREA        146 non-null    string  
 6   NOME_EMPRESA                    146 non-null    string  
 7   INDICE_SEMANA_PARTIDA_PREVISTA  146 non-null    int32   
 8   DIA_DA_SEMANA_PARTIDA_PREVISTA  146 non-null    category
dtypes: category(3), int32(2), string(4)
memory usage: 8.2 KB


In [22]:
dataframe[['FAIXA_ATRASO_PARTIDA', 'NOME_MES_PARTIDA_PREVISTA','DIA_DA_SEMANA_PARTIDA_PREVISTA' ]] = dataframe[['FAIXA_ATRASO_PARTIDA', 'NOME_MES_PARTIDA_PREVISTA','DIA_DA_SEMANA_PARTIDA_PREVISTA' ]].astype('string')

In [23]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1938 to 237113
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   FAIXA_ATRASO_PARTIDA            146 non-null    string
 1   MES_PARTIDA_PREVISTA            146 non-null    int32 
 2   NOME_MES_PARTIDA_PREVISTA       146 non-null    string
 3   SIGLA_ICAO_AEROPORTO_ORIGEM     146 non-null    string
 4   NOME_AEROPORTO_ORIGEM           146 non-null    string
 5   SIGLA_ICAO_EMPRESA_AEREA        146 non-null    string
 6   NOME_EMPRESA                    146 non-null    string
 7   INDICE_SEMANA_PARTIDA_PREVISTA  146 non-null    int32 
 8   DIA_DA_SEMANA_PARTIDA_PREVISTA  146 non-null    string
dtypes: int32(2), string(7)
memory usage: 10.3 KB


In [24]:
df_gb = dataframe.groupby(['FAIXA_ATRASO_PARTIDA', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_FAIXA_ATRASO_SAIDA=('FAIXA_ATRASO_PARTIDA', 'count')
).reset_index()


In [25]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_FAIXA_ATRASO_SAIDA']
        total_mes_anterior = linha['TOTAL_FAIXA_ATRASO_PARTIDA_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [26]:
dataframe_total = dataframe.groupby(['FAIXA_ATRASO_PARTIDA', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOO_MES=('FAIXA_ATRASO_PARTIDA', 'count')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
dataframe_total.head()

,FAIXA_ATRASO_PARTIDA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOO_MES
0,Atraso de até 30 minutos,2,fevereiro,9
1,Atraso de mais de 4 horas,2,fevereiro,2
2,No horário,2,fevereiro,135


In [28]:
fig = ff.create_table(dataframe_total)



fig.show()

- Variação de atraso Partida/Chegada total por aeroporto em relação ao mês anterior (Análise por Flag)

In [33]:
def obter_variacao_flag_atraso(
        sigla_aeroporto: str,
        flag_atraso: str,
        flag_partida_chegada: str,
        sigla_empresa: str = None,
        
    ):
    colunas = [
        'NOME_AEROPORTO_ORIGEM',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        f'MES_{flag_partida_chegada}_PREVISTA',
        f'NOME_MES_{flag_partida_chegada}_PREVISTA',
        f'DIA_DA_SEMANA_{flag_partida_chegada}_PREVISTA',
        'INDICE_SEMANA_PARTIDA_PREVISTA',
        f'FLAG_ATRASO_{flag_partida_chegada}',
        'FLAG_FIM_SEMANA_PARTIDA_PREVISTA'
    ]

    query = (
        f'SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" ' 
        f' and FLAG_ATRASO_{flag_partida_chegada} == "{flag_atraso}" '
    )
  
    if sigla_empresa is not None:
        query += f' SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}"'
        colunas.append('SIGLA_ICAO_EMPRESA_AEREA')

    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
   
    dataframe[f'NOME_MES_{flag_partida_chegada}_PREVISTA'] = dataframe[f'NOME_MES_{flag_partida_chegada}_PREVISTA'].astype('string')
    
    dataframe = dataframe.groupby(
        [
            f'FLAG_ATRASO_{flag_partida_chegada}', 
            f'NOME_MES_{flag_partida_chegada}_PREVISTA', 
            f'MES_{flag_partida_chegada}_PREVISTA'
        ]
    ).agg(
        TOTAL_ATRASO=(f'FLAG_ATRASO_{flag_partida_chegada}', 'count')
    ).reset_index().sort_values(by=[f'MES_{flag_partida_chegada}_PREVISTA'])
    dataframe['TOTAL_ATRASO_DESLOCADO'] = dataframe.groupby(f'FLAG_ATRASO_{flag_partida_chegada}')['TOTAL_ATRASO'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe[
        [f'FLAG_ATRASO_{flag_partida_chegada}', f'NOME_MES_{flag_partida_chegada}_PREVISTA']
    ] = dataframe[
        [f'FLAG_ATRASO_{flag_partida_chegada}', f'NOME_MES_{flag_partida_chegada}_PREVISTA']
    ].astype('string')
    dataframe[
        [f'MES_{flag_partida_chegada}_PREVISTA', 'TOTAL_ATRASO', 'TOTAL_ATRASO_DESLOCADO']
    ] = dataframe[
        [f'MES_{flag_partida_chegada}_PREVISTA', 'TOTAL_ATRASO', 'TOTAL_ATRASO_DESLOCADO']
    ].astype('int32')
    return dataframe

In [34]:
dataframe = obter_variacao_flag_atraso(
    flag_partida_chegada='CHEGADA',
    flag_atraso='Vôo Atrasado',
    sigla_aeroporto='SBRP'
)
dataframe

,FLAG_ATRASO_CHEGADA,NOME_MES_CHEGADA_PREVISTA,MES_CHEGADA_PREVISTA,TOTAL_ATRASO,TOTAL_ATRASO_DESLOCADO
4,Vôo Atrasado,janeiro,1,225,0
3,Vôo Atrasado,fevereiro,2,195,225
8,Vôo Atrasado,março,3,238,195
0,Vôo Atrasado,abril,4,274,238
7,Vôo Atrasado,maio,5,320,274
6,Vôo Atrasado,junho,6,303,320
5,Vôo Atrasado,julho,7,383,303
1,Vôo Atrasado,agosto,8,292,383
11,Vôo Atrasado,setembro,9,283,292
10,Vôo Atrasado,outubro,10,253,283


In [36]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=dataframe['NOME_MES_CHEGADA_PREVISTA'],
        y=dataframe['TOTAL_ATRASO'],
        name='TOTAL ATRASO ATUAL',
        textposition='auto'
    
    )
)
fig.add_trace(
    go.Scatter(
        x=dataframe['NOME_MES_CHEGADA_PREVISTA'],
        y=dataframe['TOTAL_ATRASO_DESLOCADO'],
        name='TOTAL ATRASO MÊS ANTERIOR'
    )
)

fig.show()

- Variação de atraso Partida total por aeroporto em relação ao mês anterior (Análise por Flag)

- Variação de FAIXA_ATRASO_PARTIDA por aeroporto em relação ao mês anterior 

<p> O volume de atraso de saída no fim de semana, é maior ou menor em relação aos outros dias uteis</p>

In [96]:
base_completa.columns

Index(['NOME_AEROPORTO_ORIGEM', 'CIDADE_AEROPORTO_ORIGEM',
       'UF_AEROPORTO_ORIGEM', 'SIGLA_ICAO_EMPRESA_AEREA', 'NUMERO_VOO',
       'CODIGO_TIPO_LINHA', 'SIGLA_ICAO_AEROPORTO_ORIGEM',
       'SIGLA_ICAO_AEROPORTO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL',
       'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'SIGLA_OACI',
       'NOME_AEROPORTO_DESTINO', 'CIDADE_AEROPORTO_DESTINO',
       'UF_AEROPORTO_DESTINO', 'NOME_EMPRESA', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_REAL',
       'NOME_MES_PARTIDA_REAL', 'MES_CHEGADA_PREVISTA',
       'NOME_MES_CHEGADA_PREVISTA', 'MES_CHEGADA_REAL',
       'NOME_MES_CHEGADA_REAL', 'MINUTOS_ATRASADO_PARTIDA',
       'MINUTOS_ATRASADO_CHEGADA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA',
       'INDICE_SEMANA_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_REAL',
       'INDICE_SEMANA_PARTIDA_REAL', 'DIA_DA_SEMANA_CHEGADA_REAL',
       'DIA_DA_SEMANA_CHEGADA_PREVISTA', 'INDICE_SEMANA_CHEGADA_REAL',
       'INDICE_SEMANA_CH

In [97]:
colunas = [
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_EMPRESA',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'FAIXA_ATRASO_PARTIDA',
    'FLAG_ATRASO_PARTIDA',
    'FLAG_FIM_SEMANA_PARTIDA_PREVISTA'
]

In [138]:
def obter_variacao_incidencia(
        mes: int, 
        sigla_aeroporto: str, 
        path_coluna_analise: str,
        sigla_empresa_aerea: str = None,
    ) -> pd.DataFrame:
    if sigla_empresa_aerea is None:
        query = f'SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}"  and MES_{path_coluna_analise}_PREVISTA == {mes}'
    else:
        query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" ' \
                f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa_aerea}" '\
                f' and MES_{path_coluna_analise}_PREVISTA == {mes} '
    colunas = [
        'NOME_AEROPORTO_ORIGEM',
        'SIGLA_ICAO_EMPRESA_AEREA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        f'MES_{path_coluna_analise}_PREVISTA',
        f'NOME_MES_{path_coluna_analise}_PREVISTA',
        f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA',
        f'INDICE_SEMANA_{path_coluna_analise}_PREVISTA',
        f'FAIXA_ATRASO_{path_coluna_analise}',
        f'FLAG_ATRASO_{path_coluna_analise}',
        f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
    ]
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ] = dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ].astype('string')
    dataframe = dataframe.groupby(
        [
            f'FLAG_ATRASO_{path_coluna_analise}', 
            f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
        ]
    ).agg(
        TOTAL_INCIDENCIA=(f'FLAG_ATRASO_{path_coluna_analise}', 'count')
    ).reset_index()
    dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'] = dataframe.groupby(f'FLAG_ATRASO_{path_coluna_analise}')['TOTAL_INCIDENCIA'].transform('sum')
    dataframe[f'PERCENTUAL_INCIDENCIA_{path_coluna_analise}'] = round((dataframe['TOTAL_INCIDENCIA'] * 100) / dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'], 2)
    return dataframe

In [139]:
obter_variacao_incidencia(mes=1, sigla_aeroporto='SBRP', path_coluna_analise='PARTIDA')

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA,TOTAL_INCIDENCIA_GERAL_PARTIDA,PERCENTUAL_INCIDENCIA_PARTIDA
0,Vôo Atrasado,Dia Útil,160,227,70.48
1,Vôo Atrasado,Fim de Semana,67,227,29.52
2,Vôo no horário,Dia Útil,346,378,91.53
3,Vôo no horário,Fim de Semana,32,378,8.47


In [98]:
dataframe = pd.read_parquet(caminho_base, columns=colunas)
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
0,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
1,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
2,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
3,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
4,BAURU/AREALVA,AZU,SBAE,AZUL,2,fevereiro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana


In [99]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype   
---  ------                            --------------   -----   
 0   NOME_AEROPORTO_ORIGEM             717357 non-null  string  
 1   SIGLA_ICAO_EMPRESA_AEREA          717357 non-null  string  
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       717357 non-null  string  
 3   NOME_EMPRESA                      717357 non-null  string  
 4   MES_PARTIDA_PREVISTA              717357 non-null  int32   
 5   NOME_MES_PARTIDA_PREVISTA         717357 non-null  category
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    717357 non-null  category
 7   INDICE_SEMANA_PARTIDA_PREVISTA    717357 non-null  int32   
 8   FAIXA_ATRASO_PARTIDA              717357 non-null  category
 9   FLAG_ATRASO_PARTIDA               717357 non-null  string  
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  717357 non-null  string  
dtypes: category(3), int32(2), string(6)
mem

- Incidencia de atraso total por aeroporto (Análise por Flag)

In [100]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and MES_PARTIDA_PREVISTA == 12')
# dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP" and SIGLA_ICAO_EMPRESA_AEREA == "AZU"  and MES_PARTIDA_PREVISTA == 1')
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
1992,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1993,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1994,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Domingo,6,No horário,Vôo no horário,Fim de Semana
1995,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1996,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana


In [101]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1992 to 705207
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   NOME_AEROPORTO_ORIGEM             413 non-null    string  
 1   SIGLA_ICAO_EMPRESA_AEREA          413 non-null    string  
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       413 non-null    string  
 3   NOME_EMPRESA                      413 non-null    string  
 4   MES_PARTIDA_PREVISTA              413 non-null    int32   
 5   NOME_MES_PARTIDA_PREVISTA         413 non-null    category
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    413 non-null    category
 7   INDICE_SEMANA_PARTIDA_PREVISTA    413 non-null    int32   
 8   FAIXA_ATRASO_PARTIDA              413 non-null    category
 9   FLAG_ATRASO_PARTIDA               413 non-null    string  
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  413 non-null    string  
dtypes: category(3), int32(2), string(6)
memory usage: 28.0 KB

In [102]:
dataframe[['NOME_MES_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA', 'FAIXA_ATRASO_PARTIDA']] = dataframe[['NOME_MES_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA', 'FAIXA_ATRASO_PARTIDA']].astype('string')

In [103]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1992 to 705207
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   NOME_AEROPORTO_ORIGEM             413 non-null    string
 1   SIGLA_ICAO_EMPRESA_AEREA          413 non-null    string
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       413 non-null    string
 3   NOME_EMPRESA                      413 non-null    string
 4   MES_PARTIDA_PREVISTA              413 non-null    int32 
 5   NOME_MES_PARTIDA_PREVISTA         413 non-null    string
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    413 non-null    string
 7   INDICE_SEMANA_PARTIDA_PREVISTA    413 non-null    int32 
 8   FAIXA_ATRASO_PARTIDA              413 non-null    string
 9   FLAG_ATRASO_PARTIDA               413 non-null    string
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  413 non-null    string
dtypes: int32(2), string(9)
memory usage: 35.5 KB


In [104]:
dataframe = dataframe.groupby(['FLAG_ATRASO_PARTIDA', 'FLAG_FIM_SEMANA_PARTIDA_PREVISTA' ]).agg(
    TOTAL_INCIDENCIA=('FLAG_ATRASO_PARTIDA', 'count')
).reset_index()


In [105]:
dataframe

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA
0,Vôo Atrasado,Dia Útil,151
1,Vôo Atrasado,Fim de Semana,72
2,Vôo no horário,Dia Útil,150
3,Vôo no horário,Fim de Semana,40


In [106]:
dataframe.groupby('FLAG_ATRASO_PARTIDA')['TOTAL_INCIDENCIA'].transform('sum')

0    223
1    223
2    190
3    190
Name: TOTAL_INCIDENCIA, dtype: int64

In [107]:
dataframe['TOTAL_INCIDENCIA_GERAL'] = dataframe.groupby('FLAG_ATRASO_PARTIDA')['TOTAL_INCIDENCIA'].transform('sum')

In [109]:
dataframe['PERCENTUAL_INCIDENCIA'] = round((dataframe['TOTAL_INCIDENCIA'] * 100) / dataframe['TOTAL_INCIDENCIA_GERAL'], 2)

In [110]:
dataframe

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA,TOTAL_INCIDENCIA_GERAL,PERCENTUAL_INCIDENCIA
0,Vôo Atrasado,Dia Útil,151,223,67.71
1,Vôo Atrasado,Fim de Semana,72,223,32.29
2,Vôo no horário,Dia Útil,150,190,78.95
3,Vôo no horário,Fim de Semana,40,190,21.05


In [111]:
fig = px.bar(
    dataframe,
    x='TOTAL_INCIDENCIA',
    y='FLAG_FIM_SEMANA_PARTIDA_PREVISTA',
    color='FLAG_ATRASO_PARTIDA',
    barmode='group'
)
fig.show()

<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>

<p> - Analise a FAIXA_ATRASO_CHEGADA por aeroporto e empresa por dia de semana e por mês </p>
<p> O volume de atraso no fim de semana, é maior ou menor em relação ao dia atual</p>
<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>


<p> Ranking por FAIXA_ATRASO_CHEGADA selecionando aeroporto  por mês e empresa</p>

<p> Ranking por FAIXA_ATRASO_SAIDA selecionando aeroporto em empresa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_CHEGADA selecionando aeroportoa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_PARTIFA selecionando aeroporto  por mês</p>